<a href="https://colab.research.google.com/github/marceloccs/DL-PARA-PLV/blob/main/DL_Trilha4_Atividade.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Nome:** Marcelo Custodio Cruz Silva

##**Matricula:** 92123341

---

##**Curso:** Inteligencia Artificial

##**Turma:** B

*Link do arquivo no google colabs: https://drive.google.com/file/d/1VxILG87YzaTm0EXERSHScB1Fr87sERye/view?usp=sharing*

*Link do arquivo no github: https://github.com/marceloccs/DL-PARA-PLV/blob/main/DL_Trilha4_Atividade.ipynb* 



# Trilha 4: Tarefa (Peso 3)

O reconhecimento facial é uma das aplicações mais populares em visão computacional. Nesta tarefa, você deverá desenvolver um classificador utilizando redes neurais convolucionais. Para isso, procure na Web pelo banco de dados ORL face e baixe o arquivo **ORL_faces.npz**. Baixe também o python notebook associado a esta tarefa e siga as instruções do arquivo, buscando completar os códigos em python visando à construção de uma rede neural convolucional. 

In [1]:
# Garante que o Colab utilizará o TensorFlow 2.x
%tensorflow_version 2.x
from __future__ import absolute_import, division, print_function, unicode_literals

In [2]:
import itertools
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [4]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import utils
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import datasets
from tensorflow.keras import optimizers

*Parte adicionada para descompatar e usar o arquivo que esta disponivel no GITHUB Link: https://github.com/marceloccs/DL-PARA-PLV/blob/main/ORL_faces.zip?raw=true*

In [5]:
from io import BytesIO
from zipfile import ZipFile
from urllib.request import urlopen

resp = urlopen("https://github.com/marceloccs/DL-PARA-PLV/blob/main/ORL_faces.zip?raw=true")
zipfile = ZipFile(BytesIO(resp.read()))
zipfile.namelist()

['ORL_faces.npz']

In [6]:
data = np.load(zipfile.open(zipfile.namelist()[0]))

In [7]:
X_train = data['trainX']
X_train = np.array(X_train, dtype=np.float32)/255

X_test = data['testX']
X_test = np.array(X_test, dtype=np.float32)/255

y_train = data['trainY']
y_test = data['testY']

In [8]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=.05, random_state=42)

In [9]:
im_rows = 112
im_cols = 92
im_shape = (im_rows, im_cols, 1)

1. **Reformate** os dados de treinamento, validação e teste no formato adequado para a rede neural.

*Dados do dataset:*

In [10]:
print("Shape do teste: ",X_test.shape)
print("Dados do teste: ",X_test)
print("Quantidade de possibilidades para o teste: ",X_test[0].shape)

Shape do teste:  (160, 10304)
Dados do teste:  [[0.16078432 0.18431373 0.18431373 ... 0.13725491 0.14509805 0.14901961]
 [0.17254902 0.16862746 0.1254902  ... 0.16862746 0.16862746 0.14509805]
 [0.16470589 0.16078432 0.17254902 ... 0.16470589 0.16862746 0.16078432]
 ...
 [0.39607844 0.39215687 0.40392157 ... 0.12156863 0.15686275 0.16470589]
 [0.4117647  0.42352942 0.41568628 ... 0.17254902 0.15686275 0.18431373]
 [0.44313726 0.44705883 0.43529412 ... 0.24313726 0.31764707 0.34901962]]
Quantidade de possibilidades para o teste:  (10304,)


In [11]:
print("Shape do resultado: ",y_test.shape)
print("Dados do resultado: ",(y_test))
print("Quantidade de possibilidades para o resultado: ",(np.unique(y_test).shape[0]))

Shape do resultado:  (160,)
Dados do resultado:  [ 0  0  0  0  0  0  0  0  1  1  1  1  1  1  1  1  2  2  2  2  2  2  2  2
  3  3  3  3  3  3  3  3  4  4  4  4  4  4  4  4  5  5  5  5  5  5  5  5
  6  6  6  6  6  6  6  6  7  7  7  7  7  7  7  7  8  8  8  8  8  8  8  8
  9  9  9  9  9  9  9  9 10 10 10 10 10 10 10 10 11 11 11 11 11 11 11 11
 12 12 12 12 12 12 12 12 13 13 13 13 13 13 13 13 14 14 14 14 14 14 14 14
 15 15 15 15 15 15 15 15 16 16 16 16 16 16 16 16 17 17 17 17 17 17 17 17
 18 18 18 18 18 18 18 18 19 19 19 19 19 19 19 19]
Quantidade de possibilidades para o resultado:  20


*Reshape do treino para o tamanho certo da imagem.*

In [12]:
X_train = X_train.reshape(X_train.shape[0], *im_shape)
X_test = X_test.reshape(X_test.shape[0], *im_shape)
X_valid = X_valid.reshape(X_valid.shape[0], *im_shape)

In [13]:
X_test[0].shape

(112, 92, 1)

2. **Construa** a rede neural convolucional objetivando o resultado da métrica apresentado no final deste notebook.

*Modelo para o treinamento*

In [14]:
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(X_test.shape[0]/4, kernel_size=7, activation='relu', input_shape=im_shape))
model.add(keras.layers.MaxPooling2D((2, 2)))
model.add(keras.layers.Conv2D(X_test.shape[0]/2, kernel_size=5, activation='relu'))
model.add(keras.layers.MaxPooling2D((2, 2))),
model.add(keras.layers.Conv2D(X_test.shape[0]/2, kernel_size=3, activation='relu'))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(X_test.shape[0]*2, activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(np.unique(y_test).shape[0], activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 106, 86, 40)       2000      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 53, 43, 40)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 49, 39, 80)        80080     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 24, 19, 80)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 22, 17, 80)        57680     
                                                                 
 flatten (Flatten)           (None, 29920)             0

3. **Compile** o modelo com os seguintes parâmetros:
* **Loss:** sparse_categorical_entropy;
* **Optimizer:** Adam;
* **Metrics:** accuracy.

In [15]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

4. **Ajuste** o modelo aos dados de treinamento considerando os seguintes parâmetros:
* batch_size = 512
* verbose = 2

In [16]:
historico = model.fit(X_train, y_train, epochs=25, 
          batch_size=512,verbose=2
          ,validation_data=(np.array(X_valid),np.array(y_valid))
          )

Epoch 1/25
1/1 - 12s - loss: 3.0019 - accuracy: 0.0439 - val_loss: 3.1582 - val_accuracy: 0.0000e+00 - 12s/epoch - 12s/step
Epoch 2/25
1/1 - 8s - loss: 3.0817 - accuracy: 0.0789 - val_loss: 2.9769 - val_accuracy: 0.0000e+00 - 8s/epoch - 8s/step
Epoch 3/25
1/1 - 8s - loss: 2.9773 - accuracy: 0.0482 - val_loss: 2.9901 - val_accuracy: 0.0000e+00 - 8s/epoch - 8s/step
Epoch 4/25
1/1 - 8s - loss: 2.9521 - accuracy: 0.0965 - val_loss: 3.0430 - val_accuracy: 0.0000e+00 - 8s/epoch - 8s/step
Epoch 5/25
1/1 - 8s - loss: 2.9093 - accuracy: 0.1228 - val_loss: 2.9885 - val_accuracy: 0.0000e+00 - 8s/epoch - 8s/step
Epoch 6/25
1/1 - 8s - loss: 2.8597 - accuracy: 0.1316 - val_loss: 2.8482 - val_accuracy: 0.1667 - 8s/epoch - 8s/step
Epoch 7/25
1/1 - 8s - loss: 2.7814 - accuracy: 0.1491 - val_loss: 2.7795 - val_accuracy: 0.5000 - 8s/epoch - 8s/step
Epoch 8/25
1/1 - 8s - loss: 2.6526 - accuracy: 0.3114 - val_loss: 2.6876 - val_accuracy: 0.5000 - 8s/epoch - 8s/step
Epoch 9/25
1/1 - 8s - loss: 2.4015 - accu

*Resultado*

In [17]:
scor = model.evaluate( np.array(X_test),  np.array(y_test), verbose=0)

print('test los {:.4f}'.format(scor[0]))
print('test acc {:.4f}'.format(scor[1]))

test los 0.5795
test acc 0.8188
